In [16]:
import sys
import os

sys.path.append('scripts')

import re
import json
import argparse
import itertools
from pathlib import Path
import collections
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
from matplotlib.patches import PathPatch
from matplotlib.path import get_path_collection_extents
import seaborn as sns

from rich import print, pretty
import polars as pl
from typing import  Iterable
import pretty_errors
from catppuccin import PALETTE

print(os.getcwd())

# import .scripts.ldj
from ldj import ldj
from utils import *

pretty.install()

RESULTS_DIRS = [
    Path('./experiments/communications-failure-gbpplanner-final'),
    Path('./experiments/communications-failure-lm-3-tk-5'),
]

for RESULTS_DIR in RESULTS_DIRS:
    assert RESULTS_DIR.is_dir() and RESULTS_DIR.exists()

# RESULTS_DIR = Path('./experiments/circle-experiment-lm-3-th-5')
# assert RESULTS_DIR.is_dir() and RESULTS_DIR.exists()

flavor = PALETTE.latte.colors
# num-robots-10-seed-0.json
RES = [
    re.compile(r"target-speed-(\d+)-probability-(\d+.\d+)-seed-(\d+).json"),
    re.compile(r"v0-(\d+)-failure-(\d+.\d+)-lm-3-tk-5-seed-(\d+).json"),
]

TAG = [
    "Theirs",
    "Ours",
]


/home/jens/repos/master/gbp-rs

In [8]:
# use LaTeX for text with matplotlib
sns.set_style("darkgrid")
# set background color of sns darkgrid to flavor.base.hex
plt.rcParams['axes.facecolor'] = flavor.base.hex
# set font color to flavor.text.hex
plt.rcParams['text.color'] = flavor.text.hex

font_dirs = ["./scripts/fonts/"]
# go through all fonts in the font directory and add them
for font_dir in font_dirs:
    for font in os.listdir(font_dir):
        fm.fontManager.addfont(f"{font_dir}/{font}")

prop_jbm = fm.FontProperties(fname='./scripts/fonts/JetBrainsMonoNerdFontMono-Regular.ttf')
prop = fm.FontProperties(fname='./scripts/fonts/STIXTwoText-VariableFont_wght.ttf')

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": prop.get_name(),
    # "font.family": "stix",
    # "font.sans-serif": prop.get_name(),
    "mathtext.fontset": "stix",
    # "text.latex.preamble": r"\usepackage{fontenc}\usepackage{fontspec}\setmainfont{JetBrainsMonoNerdFontMono-Regular}",
})

print(prop.get_name())

colors = [(flavor.lavender.hex, 1.0), (flavor.yellow.hex, 0.3), (flavor.peach.hex, 0.3)]

STIX Two Text

In [55]:

def flatten(lst: Iterable) -> list:
    return list(itertools.chain.from_iterable(lst))

def process_file_(file, re, tag):
    # print(f"Processing {file} with {re}")
    match = re.match(file.name)
    assert match is not None
    target_speed = float(match.group(1))
    failure_rate = float(match.group(2))
    seed = int(match.group(3))

    with open(file, 'r') as file:
        data = json.load(file)

    time_to_completion_of_each_robot: list[float] = []

    collisions = len(data['collisions']['robots']) if tag == "Ours" else 0

    for _, robot_data in data['robots'].items():
        mission = robot_data['mission']
        
        t_start: float = mission['started_at']
        t_final: float = mission['finished_at'] if mission['finished_at'] else mission['duration'] + t_start
        time_to_completion_of_each_robot.append(t_final - t_start)

    # makespan: float = data['makespan']
    makespan: float = max(time_to_completion_of_each_robot)
    # print(f"{makespan=}")
    return target_speed, failure_rate, makespan, collisions

In [56]:
df = pl.DataFrame(schema={
    "tag": pl.String,
    "target_speed": pl.Float64,
    "failure_rate": pl.Float64,
    "makespan": pl.Float64,
    "collisions": pl.Int64,
})

for RESULTS_DIR, RE, T in zip(RESULTS_DIRS, RES, TAG):
    # with ProcessPoolExecutor() as executor:
    #     results = executor.map(process_file_, RESULTS_DIR.glob('*.json'))

    print(f"Processing {RESULTS_DIR=}")

    results = [process_file_(file, RE, T) for file in RESULTS_DIR.glob('*.json')]

    # Aggregate results in a single-threaded manner to avoid data
    # aggregated_data_distance_travelled: dict[int, list[float]] = collections.defaultdict(list)
    aggregated_data_makespan: dict[int, list[float]] = collections.defaultdict(list)
    # aggregated_data_ldj: dict[int, list[float]] = collections.defaultdict(list)

    for speed, failure_rate, makespan, collisions in results:
        # print(f"{comms_radius=} {makespan=}")
        # aggregated_data_distance_travelled[comms_radius].extend(distance_travelled_for_each_robot)
        # aggregated_data_makespan[failure_rate].append(makespan)
        # aggregated_data_ldj[comms_radius].extend(ldj_for_each_robot)

        df2 = pl.DataFrame({
            "tag": [T],
            "target_speed": [speed],
            "failure_rate": [failure_rate],
            "makespan": [makespan],
            "collisions": [collisions],
        })

        df = df.extend(df2)

    # data_distance = [aggregated_data_distance_travelled[key] for key in sorted(aggregated_data_distance_travelled.keys())]
    # labels_distance = sorted(aggregated_data_distance_travelled.keys())

    # data_distance_dict = dict(zip(labels_distance, data_distance))

    # data.append(
    #     {
    #         # 'distance': data_distance_dict,
    #         # 'labels_distance': labels_distance,
    #         'makespan': aggregated_data_makespan,
    #         # 'ldj': aggregated_data_ldj
    #     }
    # )


Processing RESULTS_DIR=PosixPath('experiments/communications-failure-gbpplanner-final')

Processing RESULTS_DIR=PosixPath('experiments/communications-failure-lm-3-tk-5')

In [58]:
df_theirs = df.filter(df['tag'] == "Theirs")
print(df_theirs.filter(df_theirs['target_speed'] == 10).sort("failure_rate"))
print(df_theirs.filter(df_theirs['target_speed'] == 15).sort("failure_rate"))

shape: (10, 5)
┌────────┬──────────────┬──────────────┬────────────┬────────────┐
│ tag    ┆ target_speed ┆ failure_rate ┆ makespan   ┆ collisions │
│ ---    ┆ ---          ┆ ---          ┆ ---        ┆ ---        │
│ str    ┆ f64          ┆ f64          ┆ f64        ┆ i64        │
╞════════╪══════════════╪══════════════╪════════════╪════════════╡
│ Theirs ┆ 10.0         ┆ 0.0          ┆ 86.300003  ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.1          ┆ 146.300003 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.2          ┆ 140.199997 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.3          ┆ 163.699997 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.4          ┆ 157.400009 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.5          ┆ 231.400009 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.6          ┆ 257.200012 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.7          ┆ 260.899994 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.8          ┆ 362.100006 ┆ 0          │
│ Theirs ┆ 10.0         ┆ 0.9          ┆ 938.900024 ┆ 0          │
└────────┴──────────────┴──────────────┴────────────┴────────────┘

shape: (10, 5)
┌────────┬──────────────┬──────────────┬────────────┬────────────┐
│ tag    ┆ target_speed ┆ failure_rate ┆ makespan   ┆ collisions │
│ ---    ┆ ---          ┆ ---          ┆ ---        ┆ ---        │
│ str    ┆ f64          ┆ f64          ┆ f64        ┆ i64        │
╞════════╪══════════════╪══════════════╪════════════╪════════════╡
│ Theirs ┆ 15.0         ┆ 0.0          ┆ 238.600006 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.1          ┆ 203.199997 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.2          ┆ 329.800018 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.3          ┆ 231.100006 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.4          ┆ 220.800003 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.5          ┆ 221.5      ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.6          ┆ 328.700012 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.7          ┆ 505.600006 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.8          ┆ 799.600037 ┆ 0          │
│ Theirs ┆ 15.0         ┆ 0.9          ┆ 864.600037 ┆ 0          │
└────────┴──────────────┴──────────────┴────────────┴────────────┘

In [61]:
df_ours = df.filter(df['tag'] == "Ours")
df_ours_10 = df_ours.filter(df_ours['target_speed'] == 10.0)
df_ours_outlier_free = df_ours_10.filter(df_ours_10['makespan'] < 1000)
print(df_ours_outlier_free.sort("failure_rate").group_by("failure_rate").agg(pl.mean("makespan")))
print(df_ours_outlier_free.sort("failure_rate").group_by("failure_rate").agg(pl.mean("collisions")))

df_ours_15 = df_ours.filter(df_ours['target_speed'] == 15.0)
df_ours_outlier_free = df_ours_15.filter(df_ours_15['makespan'] < 1000)
print(df_ours_outlier_free.sort("failure_rate").group_by("failure_rate").agg(pl.mean("makespan")))
print(df_ours_outlier_free.sort("failure_rate").group_by("failure_rate").agg(pl.mean("collisions")))

shape: (8, 2)
┌──────────────┬──────────┐
│ failure_rate ┆ makespan │
│ ---          ┆ ---      │
│ f64          ┆ f64      │
╞══════════════╪══════════╡
│ 0.0          ┆ 21.125   │
│ 0.1          ┆ 24.24    │
│ 0.2          ┆ 27.1     │
│ 0.3          ┆ 29.06    │
│ 0.4          ┆ 33.16    │
│ 0.5          ┆ 39.84    │
│ 0.6          ┆ 48.4     │
│ 0.7          ┆ 60.82    │
└──────────────┴──────────┘

shape: (8, 2)
┌──────────────┬────────────┐
│ failure_rate ┆ collisions │
│ ---          ┆ ---        │
│ f64          ┆ f64        │
╞══════════════╪════════════╡
│ 0.0          ┆ 0.5        │
│ 0.1          ┆ 1.8        │
│ 0.2          ┆ 6.8        │
│ 0.3          ┆ 11.0       │
│ 0.4          ┆ 18.4       │
│ 0.5          ┆ 33.0       │
│ 0.6          ┆ 39.6       │
│ 0.7          ┆ 51.8       │
└──────────────┴────────────┘

shape: (8, 2)
┌──────────────┬──────────┐
│ failure_rate ┆ makespan │
│ ---          ┆ ---      │
│ f64          ┆ f64      │
╞══════════════╪══════════╡
│ 0.0          ┆ 20.3     │
│ 0.1          ┆ 23.8     │
│ 0.2          ┆ 26.9     │
│ 0.3          ┆ 29.6     │
│ 0.4          ┆ 33.5     │
│ 0.5          ┆ 40.2     │
│ 0.6          ┆ 48.78    │
│ 0.7          ┆ 68.2     │
└──────────────┴──────────┘

shape: (8, 2)
┌──────────────┬────────────┐
│ failure_rate ┆ collisions │
│ ---          ┆ ---        │
│ f64          ┆ f64        │
╞══════════════╪════════════╡
│ 0.0          ┆ 1.2        │
│ 0.1          ┆ 1.8        │
│ 0.2          ┆ 6.2        │
│ 0.3          ┆ 11.2       │
│ 0.4          ┆ 18.2       │
│ 0.5          ┆ 36.0       │
│ 0.6          ┆ 41.6       │
│ 0.7          ┆ 55.6       │
└──────────────┴────────────┘

In [ ]:
# for each comms radius, [20, 40, 60, 80], calculate mean makespan, distance travelled, and ldj

# for i, d in enumerate(data):

#     makespan_means = dict([(key, np.mean(d['makespan'][key])) for key in sorted(d['makespan'].keys())])
#     # distance_means = dict([(key, np.mean(d['distance'][key])) for key in sorted(d['distance'].keys())])
#     # ldj_means = dict([(key, np.mean(d['ldj'][key])) for key in sorted(d['ldj'].keys())])

#     print(f"{RESULTS_DIRS[i].name}")
#     print(f"{makespan_means=}")
#     # print(f"{distance_means=}")
#     # print(f"{ldj_means=}")
    